In [ ]:
%%time
import numpy as np
import scanpy as sc
import os
import pandas as pd
import bbknn
import matplotlib.pyplot as plt
import seaborn as sns
sc.settings.verbosity = 4                
sc.settings.set_figure_params(80)                 
sc.settings.file_format_figures = 'pdf'  
sc.settings.savefigs = False            
use_first_n_samples = 0
full_sparse = False


After creating gProfiler csv files 
- For the conditions DE of monocytes - Created by top 100 DE genes upregulated for each condition and sorted by Q values 
- For the 3 states - Created with the three sets of state-DE genes from unstimulated monocytes and the two dsRNA-stimulated monocytes

In [ ]:
def go_terms_unique(main_path, source =['GO:BP','GO:CC','KEGG','REAC'],states = ['group1', 'group2', 'group3']):
    gp_dfs = {}
    gp_terms = {}
    all_terms = []
    for group in states:
        gp_dfs[group] = pd.read_csv(main_path+group+".csv",index_col=0)
        gp_terms[group] = list(gp_dfs[group].term_id.values)
        for i in gp_terms[group]:
            all_terms.append(i)
    all_terms_unique = [i for i in all_terms if all_terms.count(i) == 1]

    gp_dfs_unique={}
    unique_terms = {}
    for group in gp_terms.keys():
        unique_terms[group] =[term for term in gp_terms[group] if term in all_terms_unique] 
        gp_dfs_unique[group] = gp_dfs[group][gp_dfs[group].term_id.isin(unique_terms[group])]
        gp_dfs_unique[group].sort_values(by='adjusted_p_value',inplace=True)
        gp_dfs_unique[group]['-log10(Qvalue)'] = -np.log10(gp_dfs_unique[group]['adjusted_p_value'])
        gp_dfs_unique[group] = gp_dfs_unique[group][gp_dfs_unique[group].index.isin(source)]
    for group in gp_dfs_unique.keys():
        sns.set(rc={'figure.figsize':(11.7,8.27)})
        sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})
        fig = sns.barplot(x="-log10(Qvalue)", y='term_name',data=gp_dfs_unique[group].iloc[:20], color='#6D01FA', palette='RdBu').set_title(group+'_'+','.join(source))
        plt.savefig(group +'.pdf',bbox_inches='tight',transparent=True, format="pdf")
        plt.show()
    return gp_dfs_unique

In [ ]:
#Some genes annotated from NCBI https://www.ncbi.nlm.nih.gov/genome/gdv/browser/genome/?id=GCF_001466805.2
locs = {'LOC107499671':'MT2A','LOC107502043':'DDX3X','LOC107511263':'IFI27L2A','LOC107517540':'H2-Q10','LOC107499653':'KRTAP5-9','LOC107518587':'IFN4A','LOC107510717':'PTPN11','LOC107504525':'H4C1','LOC107504040':'H3C1','LOC107503981':'H3C1'}